In [3]:
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne import Epochs, pick_types, find_events
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

from mne import Epochs, pick_types, find_events
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne import events_from_annotations
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from sklearn.model_selection import train_test_split



In [ ]:

print(__doc__)
tmin, tmax = -1., 4.
event_id = dict(hands=2, feet=3)
subject = 1
runs = [6, 10, 14]  # motor imagery: hands vs feet

raw_fnames = eegbci.load_data(subject, runs)
raw_files = [read_raw_edf(f, preload=True) for f in raw_fnames]
raw = concatenate_raws(raw_files)

# strip channel names of "." characters
raw.rename_channels(lambda x: x.strip('.'))

# Apply band-pass filter
raw.filter(7., 30.)


# Convert annotations to events
events, _ = events_from_annotations(raw)
picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')
epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True)
epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
epochs_test = epochs.copy().crop(tmin=2., tmax=3.)  # Adjust time window for test data
labels_train = epochs_train.events[:, -1] - 2
labels_test = epochs_test.events[:, -1] - 2
# Data normalization
X_train = epochs_train.get_data()
X_test = epochs_test.get_data()
mean, std = X_train.mean(), X_train.std()
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# Data augmentation
def augment_data(X):
    return np.flip(X, axis=-1)  # Horizontal flip

X_train_augmented = augment_data(X_train)
X_train = np.concatenate((X_train, X_train_augmented), axis=0)
labels_train = np.concatenate((labels_train, labels_train), axis=0)
import numpy as np

# Assuming X_train and X_test are your input data arrays
X_train = X_train.reshape(-1, 64, 161, 1)  # Reshape for one channel
X_test = X_test.reshape(-1, 64, 161, 1)
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(labels_train, dtype=torch.long)
y_test_tensor = torch.tensor(labels_test, dtype=torch.long)
# Define a simple fully connected graph
num_nodes = X_train_tensor.size(1)  # Number of electrodes
edge_index = torch.tensor([(i, j) for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()
edge_attr = None  # No edge attributes
# Define GCNN architecture
class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(X_train_tensor.size(-1), 32)
        self.conv2 = GCNConv(32, 64)
        self.fc = nn.Linear(64, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = global_mean_pool(x, torch.zeros(x.size(0), dtype=torch.long))
        x = self.fc(x)
        return torch.sigmoid(x)

# Instantiate the model
model = GCN()
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
# Prepare data loader
train_data = Data(x=X_train_tensor, edge_index=edge_index, edge_attr=edge_attr, y=y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)




In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, X_tensor, edge_index, y_tensor):
        self.X_tensor = X_tensor
        self.edge_index = edge_index
        self.y_tensor = y_tensor
        
    def __len__(self):
        return len(self.y_tensor)
    
    def __getitem__(self, idx):
        sample = {
            'x': self.X_tensor[idx],
            'edge_index': self.edge_index,
            'y': self.y_tensor[idx]
        }
        return sample

# Create a dataset instance
train_dataset = CustomDataset(X_train_tensor, edge_index, y_train_tensor)

# Create a data loader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [15]:
print("X_train_tensor shape:", X_train_tensor.shape)
print("edge_index shape:", edge_index.shape)
print("y_train_tensor shape:", y_train_tensor.shape)


X_train_tensor shape: torch.Size([90, 64, 161, 1])
edge_index shape: torch.Size([2, 4032])
y_train_tensor shape: torch.Size([90])


In [14]:
# Prepare data loader
train_data = Data(x=X_train_tensor, edge_index=edge_index, edge_attr=edge_attr, y=y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)


/Users/heerkubadia/Library/Python/3.9/lib/python/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
train_loader
print(len(train_loader))
print(len(train_data))

for data in train_loader:
  
    print(data)

            

In [19]:
for data in train_loader:
    print(data['x'].shape, data['edge_index'].shape, data['y'].shape)


torch.Size([32, 64, 161, 1]) torch.Size([32, 2, 4032]) torch.Size([32])
torch.Size([32, 64, 161, 1]) torch.Size([32, 2, 4032]) torch.Size([32])
torch.Size([26, 64, 161, 1]) torch.Size([26, 2, 4032]) torch.Size([26])


In [24]:
# Training the GCN model
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()  # Set the model to train mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for data in train_loader:
            
            optimizer.zero_grad()
            out = model(data.x, data.edge_index)
            
            loss = criterion(out, data.y.view(-1, 1).float())
            loss.backward()
            optimizer.step()
            
            # Compute training accuracy
            predicted = (out > 0.5).float()
            correct += (predicted == data.y.view(-1, 1).float()).sum().item()
            total += data.y.size(0)

            running_loss += loss.item() * data.y.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = correct / total
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=10)


AttributeError: 'dict' object has no attribute 'x'